In [ ]:
#DisGeNet_dis_gene.txt
dis_gene_edges_index = set()
dis_gene_edges_name = set()
genes_num=0
dis_num=0
with open(dis_ge_file, 'r') as f:
    for line in f:
        lines += 1
        words = line.split()
        id1 = words[0]
        if id1 in disNI:
            index_id1 = disNI[id1]
        else:
            dis_num+=1
            index_id1 = index
            disNI[id1] = index_id1
            disIN[index_id1] = id1
            index += 1

        id2 = words[1]
        if id2 in geneNI:
            index_id2 = geneNI[id2]
        else:
            index_id2 = index
            geneNI[id2] = index_id2
            geneIN[index_id2] = id2
            index += 1
        dis_gene_edges_index.add((index_id1, index_id2))
        dis_gene_edges_name.add((id1, id2))

print("diseases: {}\nedges: {}\ndisNI: {}\ndisNI: {}\nlines:{}".format(
    dis_num, len(dis_gene_edges_index), len(disNI), len(disIN), lines))


# save_edges(edges_filename,pos_edges,neg_edges)
# #提取正例
# pos_edges=list(dis_gene_edges_index)
# sample_ratio=1
# neg_nums=(int)(sample_ratio*len(pos_edges))
# neg_edges=[]
# disease_nums=len(dis_nodes)
# gene_nums=len(gene_nodes)
# count=0
# while count<neg_nums:
# #     index1=random.randint(0,disease_nums)
# #     index2=random.randint(0,gene_nums)
# #     edge=(dis_nodes[index1],gene_nodes[index2])
#     index1=random.choice(dis_nodes)
#     index2=random.choice(gene_nodes)
#     edge=(index1,index2)
#     if edge not in pos_edges and edge not in neg_edges:
#         neg_edges.append(edge)
#         count+=1


# with open('train_edges.txt', 'w') as f:
#     for node1,node2 in pos_edges:
#         temp=str(node1)+" "+str(node2)+" "+str(1)+"\n"
#         f.write(temp)
#     for node1,node2 in neg_edges:
#         temp=str(node1)+" "+str(node2)+" "+str(0)+"\n"
#         f.write(temp)




# # dis_gene_network.write
# input_path="network_edgelist/dis_gene_network_edgelist.txt"
# dg_network=Graph()
# dg_network.read_adjlist(filename=input_path)
# #网络表征模型
# node2vec_dis_gene = openne.node2vec.Node2vec(graph=dg_network, path_length=walk_length,
#                                   num_paths=number_walks, dim=representation_size,
#                                   workers=workers, p=p, q=q, window=window_size)
# node2vec_dis_gene.save_embeddings("embeddings/dg_network_node2vec.txt")

# node2vec_dg=node2vec_dis_gene.vectors
# print(type(node2vec_dg))
# nodes=np.array(list(node2vec_dg.keys()))
# dg_network_node2vec_vecs=np.array(list(node2vec_dg.values()))
# # for node, vec in disease_network_node2vec.items():
# #     diseases.append(node)
# #     vecs.append(vecs)
# print(nodes.shape)
# print(dg_network_node2vec_vecs.shape)



#python -m openne --method node2vec --label-file data/blogCatalog/bc_labels.txt 
#--input data/blogCatalog/bc_adjlist.txt --graph-format adjlist --output vec_all.txt --q 0.25 --p 0.25
representation_size=128 #表征向量的长度
number_walks=40  #每个节点随机游走序列数量
walk_length=10   #每个随机游走序列的长度
workers=8        #并行数量
window_size=10   #skip-gram提取词的上下文数量
#node2vec参数
q=0.25
p=0.25


# deepwalk_gene = openne.node2vec.Node2vec(graph=gene_network, path_length=walk_length,
#                                   num_paths=number_walks, dim=representation_size,
#                                   workers=workers, window=window_size, dw=True)
# deepwalk_gene.save_embeddings("embeddings/gene_network_deepwalk.txt")

# dis_gene_network.write
input_path="network_edgelist/all_network_edgelist.txt"
all_network=Graph()
all_network.read_adjlist(filename=input_path)
#网络表征模型
node2vec_all = openne.node2vec.Node2vec(graph=all_network, path_length=walk_length,
                                  num_paths=number_walks, dim=representation_size,
                                  workers=workers, p=p, q=q, window=window_size)
node2vec_all.save_embeddings("embeddings/all_network_node2vec.txt")


#边的特征
#提取正例特征
pos_features=[]
label=[]
for disease_id,gene_id in pos_edges:
    pos_features.append(np.hstack((node2vec_dg[str(disease_id)],node2vec_dg[str(gene_id)])))
    label.append(1)

#提取负例特征
neg_features=[]
for disease_id,gene_id in neg_edges:
    neg_features.append(np.hstack((node2vec_dg[str(disease_id)],node2vec_dg[str(gene_id)])))
    label.append(0)
train_features=pos_features+neg_features
print('{}:{}'.format(len(pos_features),len(pos_features[0])))
print('{}:{}'.format(len(neg_features),len(neg_features[0])))
print(len(label))
print(np.array(train_features).shape)
#基于边的特征
feature_columns=['n2v_'+str(i) for i in range(representation_size*2)]
train_data=pd.DataFrame(train_features,columns=feature_columns)
train_data['label']=label
train_data.to_csv("train_data/dg128_train_data.csv",index = False)
train_data.head()


# from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

train_data = shuffle(train_data)
train_data['label'][:10]
print("begin")
#基于边的
feature_columns=['n2v_'+str(i) for i in range(representation_size*2)]
X = train_data[feature_columns]
Y = train_data['label']
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model no training data
print('begin:')
clf = RandomForestClassifier(n_estimators=20)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)
# model = XGBClassifier()
# model.fit(X_train, y_train)
# make predictions for test data
# y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
data_path="train_data/allnet_node2vec128_train_data.csv"
test_size = 0.2
train_data=pd.read_csv(data_path)#,index_col=0
train_data = shuffle(train_data)

# print(train_data.shape)
# train_data.head()

# train_data = shuffle(train_data)
# train_data['label'][:10]


# representation_size=128 #表征向量的长度
# number_walks=40  #每个节点随机游走序列数量
# walk_length=10   #每个随机游走序列的长度
# workers=8        #并行数量
# window_size=10   #skip-gram提取词的上下文数量
# #node2vec参数
# q=0.25
# p=0.25

#ALLNET
representation_size=128 #表征向量的长度
print("begin")
#基于边的
feature_columns=[str(i) for i in range(representation_size*2)]
X = train_data[feature_columns]
Y = train_data['label']
# split data into train and test sets
seed = 7
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
print("end")


# fit model no training data
print('begin:')
clf_rf = RandomForestClassifier(n_estimators=100)
clf_rf.fit(X_train, y_train)
y_pred = clf_rf.predict(X_test)
y_prob = clf_rf.predict_proba(X_test)
# model = XGBClassifier()
# model.fit(X_train, y_train)
# make predictions for test data
# y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))





In [3]:
path="file_"
ratio_list=[0.5,0.8,1.0,1.2,1.4,1.6]
for ratio in ratio_list:
    print(path+str(ratio)+'.txt')

file_0.5.txt
file_0.8.txt
file_1.0.txt
file_1.2.txt
file_1.4.txt
file_1.6.txt


In [ ]:
edges_filename="network_edgelist/train_edges_1.0.txt"
pos_edges,neg_edges=read_edges(edges_filename)

embedding_file="embeddings/all_network_node2vec128.txt"
count=0
embeddings={}
# nodes=set()
with open(embedding_file, 'r') as f:
    first_line=f.readline()
    node_nums,embedding_dimension=first_line.split()
    print("{} {}".format(node_nums,embedding_dimension))
    for line in f:
        line=line.split()
        embeddings[line[0]]=line[1:]
        count+=1
#         node=line[0]
#         embedding=line[1:]
#         print(embedding)
#         nodes.add(node)
#         print(node)

# print(embeddings['31812'])
print(count)

#边的特征
#提取正例特征
pos_features=[]
label=[]
for disease_id,gene_id in pos_edges:
    pos_features.append(np.hstack((embeddings[str(disease_id)],embeddings[str(gene_id)])))
    label.append(1)

#提取负例特征
neg_features=[]
for disease_id,gene_id in neg_edges:
    neg_features.append(np.hstack((embeddings[str(disease_id)],embeddings[str(gene_id)])))
    label.append(0)
train_features=pos_features+neg_features

In [ ]:
print("begin")
print('{}:{}'.format(len(pos_features),len(pos_features[0])))
print('{}:{}'.format(len(neg_features),len(neg_features[0])))
print(len(label))
print(np.array(train_features).shape)

In [ ]:
print("begin")
#基于边的特征
feature_columns=['n2v_'+str(i) for i in range(representation_size*2)]
train_data=pd.DataFrame(train_features,columns=feature_columns)
train_data['label']=label
train_data.to_csv("train_data/allnet128_train_data.csv",index = False)
train_data.head()

In [ ]:

"""
功能: 随机提取正负例
输入: 
    edges_set:list:为网络中已有连边的集合,一般为疾病-基因网络的边表
    sample_ratio:正负例比例
    save_path:保存路径
返回: 提取后的正负用例pos_edges:list,neg_edges:list
"""
def extract_edges_random(edges_set,sample_ratio,save_path):
#     dis_nodes,gene_nodes=getNodelist(dis_gene_file,disNI,geneNI)
    #提取正例
    pos_edges=list(edges_set)
    neg_nums=(int)(sample_ratio*len(pos_edges))
    neg_edges=[]
    dis_nodes=set()#list(set(pos_edges[:][0]))
    gene_nodes=set()#list(set(pos_edges[:][1]))
    for edge in pos_edges:
        dis_nodes.add(edge[0])
        gene_nodes.add(edge[1])
    dis_nodes=list(dis_nodes)
    gene_nodes=list(gene_nodes)
    count=0
    while count<neg_nums:
        index1=random.choice(dis_nodes)
        index2=random.choice(gene_nodes)
        edge=(index1,index2)
        if edge not in pos_edges and edge not in neg_edges:
            neg_edges.append(edge)
            count+=1
    
    save_edges(save_path,pos_edges,neg_edges)
    return pos_edges,neg_edges


#input: 
#     edges_set:list 为网络中已有连边的集合,一般为疾病-基因网络的边表
#     sample_ratio 正负例比例
#output: pos_edges:list,neg_edges:list
#all_edges
def extract_edges(edges,sample_nums,save_path):
#     dis_nodes,gene_nodes=getNodelist(dis_gene_file,disNI,geneNI)
    network = nx.Graph() # 创建无向图
    network.add_edges_from(list(edges))
    #提取正例
    #G.degree(1)
    #pos_edges=edges
    neg_nums=sample_nums
    neg_edges=[]
    dis_nodes=set()#list(set(pos_edges[:][0]))
    gene_nodes=set()#list(set(pos_edges[:][1]))
    for edge in edges:
        dis_nodes.add(edge[0])
        gene_nodes.add(edge[1])
    dis_nodes=list(dis_nodes) 
    gene_nodes=list(gene_nodes)
    count=0
    while count<neg_nums:
        index1=random.choice(dis_nodes)
        index2=random.choice(gene_nodes)
        edge=(index1,index2)
        if edge not in pos_edges and edge not in neg_edges:
            neg_edges.append(edge)
            count+=1
    
    save_edges(save_path,pos_edges,neg_edges)
    return pos_edges,neg_edges

In [ ]:
nx.write_edgelist(dg_network, "network_edgelist/dg_network_edgelist.txt", delimiter=' ',data=False)
nx.write_edgelist(dgs_network, "network_edgelist/dgs_network_edgelist.txt", delimiter=' ',data=False)
nx.write_edgelist(dgp_network, "network_edgelist/dgp_network_edgelist.txt", delimiter=' ',data=False)
nx.write_edgelist(dgpo_network, "network_edgelist/dgpo_network_edgelist.txt", delimiter=' ',data=False)
nx.write_edgelist(dgsp_network, "network_edgelist/dgsp_network_edgelist.txt", delimiter=' ',data=False)
nx.write_edgelist(all_network, "network_edgelist/all_network_edgelist.txt", delimiter=' ',data=False)


all_network = nx.Graph() # 创建无向图
all_network.add_edges_from(list(all_edges))
# all_network=degree_graph(all_network)
gene_go_network = nx.Graph() # 创建无向图
gene_go_network.add_edges_from(list(gene_go_edges_index))
# gene_go_degree=degree_graph(gene_go_network)
dis_symp_network = nx.Graph() # 创建无向图
dis_symp_network.add_edges_from(list(dis_symp_edges_index))
# dis_symp_degree=degree_graph(dis_symp_network)
dis_gene_network = nx.Graph() # 创建无向图
dis_gene_network.add_edges_from(list(dis_gene_edges_index))
# dis_gene_degree=degree_graph(dis_gene_network)
ppi_network = nx.Graph() # 创建无向图
ppi_network.add_edges_from(list(ppi_edges_index))
# ppi_degree=degree_graph(ppi_network)

nx.write_adjlist(dis_gene_network, "network_edgelist/dis_gene_network_adjlist.txt")
nx.write_adjlist(ppi_network, "network_edgelist/ppi_network_adjlist.txt")
nx.write_adjlist(dis_symp_network, "network_edgelist/dis_symp_network_adjlist.txt")
nx.write_adjlist(gene_go_network, "network_edgelist/gene_go_network_adjlist.txt")
nx.write_adjlist(all_network, "network_edgelist/all_network_adjlist.txt")